In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.vectorstores import InMemoryVectorStore

loader = CSVLoader(file_path="history_test.csv", source_column="계정과목")
docs = loader.load()  

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = InMemoryVectorStore.from_documents(docs, embedding=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

prompt_template = """
아래는 회사의 과거 유사 거래 사례와 그에 대한 계정 과목 분류 내역입니다:
{context}

아래의 새로운 거래 내역을 보고, 과거 사례를 참고하여
어떤 계정 과목으로 분류하면 좋을지 판단해주세요:

새로운 거래: {question}

최대한 간단하게, "계정 과목"만 요약해서 결과로 제시해 주세요.
불필요한 부가설명은 생략하고, 계정 과목명만 한국어로 1개만 출력 바랍니다.
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

def classify_new_transactions(new_csv, output_csv):
    df = pd.read_csv(new_csv)
    results = []

    for idx, row in df.iterrows():
        transaction_text = str(row)
        print(transaction_text)
        answer = chain.invoke(input=transaction_text)
        results.append(answer.content.strip())

    df["계정과목"] = results
    df.to_csv(output_csv, index=False)
    print(f"[완료] '{output_csv}'에 결과를 저장했습니다.")

if __name__ == "__main__":
    classify_new_transactions("input_test.csv", "result.csv")